# <font color=blue>С/С++ vs Python</font>

### Цели работы

1. Получить представление о способах совместного использования Python и C/C++.
2. Измерить эффективность использования вычислительных ресурсов программами, реализованными на C/C++, Python и с помощью методов, комбинирующих C/C++ и Python. 

### Методы

1. C/C++

2. Python

3. Numpy -- библиотека Python для операций с массивами. 

4. Cython -- инструмент для компиляции программ на Python в исполняемые файлы. В код Python можно делать вставки кода на C.

5. `ctypes` -- библиотека Python для подключения библиотек C/C++.

6. JIT -- компиляция кода Python в машинный код в процессе выполнения программы.

7. Python C/C++ extensions -- реализация модулей Python на C/C++.

8. При желании можете попробовать и другие инструменты, например `boost`, CFFI.

### Задачи.

Примените перечисленные методы для решения задач:

- умножение матриц,

- сортировка пузырьком,

- функция Аккермана.

### Что должно получится
7 программ (по одной на каждый из методов), в которых будет замеряться время, затраченное на вычисления для каждой из задач.


## <font color=green>Измерение времени выполнения кода Python</font>

Для измерения времени, затраченного на выполнение кода, рекомендуется модуль [`timeit`](https://docs.python.org/3/library/timeit.html). Для целей этой работы подойдет функция `timeit.timeit()`.

Исследуемый участок кода передается через параметр `stmt` (statement) в виде строки. Функция `timeit.timeit()` возвращает **общее** время, которое было затрачено на `number` повторных выполнений кода `stmt`. Код `stmt` выполняется в изолированном окружении, и поэтому при вызове `timeit.timeit()` надо указать, какие переменные и функции будут доступны в коде `stmt`. Этой цели служат параметры `setup` и `globals`. 

Параметр `globals` -- словарь, с помощью которого инициализируется пространство имен для кода `stmt`. Ключи словаря `globals` -- это строки, которые служат именами переменных и функций в коде `stmt`. См. пример 1.

Параметр `setup` -- строка с кодом, который выполняется функцией `timeit.timeit()` **1 раз** (до начала измерения времени). Переменные и функции определенные в коде `setup` доступны в коде `stmt`.

### Пример 1. Функция  `timeit.timeit()`

In [ ]:
import timeit


def list_mul(a, b):
    """Multiplies lists elementwise."""
    c = []
    for i in range(len(a)):
        c.append(a[i] * b[i])
    return c


setup = """
import random
a = []
b = []
for i in range(1000):
    a.append(random.randint(0, 10**6))
    b.append(random.randint(0, 10**6))
"""
stmt = "c = list_mul(a, b)"
n = 1000

t = timeit.timeit(
    # stmt расшивровывается, как statement
    # Функция `timeit.timeit()` возвращает время,
    # затраченное на `number` повторных
    # выполнений кода из переменной `stmt`.
    stmt=stmt,  
    
    # Код из `setup` выполняется 1 раз в начале работы функции `timeit.timeit()`.
    # Параметр `setup` используется для инициализации переменных, необходимых `stmt`.
    setup=setup, 
    
    # `number` -- число повторных выполнений `stmt`.
    number=n,
    
    # Объекты, не определенные в `setup`, можно передать через параметр `globals`
    # Обратите внимание, что функция `list_mul()` не вызывается. Если функцию вызвать,
    # то в `globals` попадет значение, возвращенное функцией, а не сама функция.
    globals={"list_mul": list_mul}
)

print("Average time:", t / n)

## <font color=green>Умножение матриц</font>

Напишите функцию для умножения вещественных матриц. При постарайтесь использовать матрицы большого размера, например $1000 \times 1500$ и $1500 \times 2000$.

Тип `float` в Python -- соответствует типу `double` в  С.

В библиотеке Numpy есть метод для умножения матриц `numpy.matmul()` с эффективной реализацией операции умножения. В `numpy.matmul()` используются функции из библиотеки BLAS (Basic Linear Algebra Subprograms), дающие колоссальный прирост производительности. В связи с этим:

- для сравнения Numpy с чистым Python нужно написать наивное умножение матриц на Numpy;

- для сравнения `numpy.matmul()` с C/C++, Cython, `ctypes`, C/C++ extensions в коде C/C++ надо использовать функцию `dgemm()` (**d**ouble precision **ge**neral **m**atrix **m**ultiply) из библиотеки BLAS. В зависимости от пакета, с которым поставлен BLAS, `dgemm()` может называться по-разному: `cblas_dgemm()`, `gsl_blas_dgemm()` и т.д..

Случайная матрица в `numpy` создается с помощью 

In [ ]:
import numpy as np

a = np.random.rand(1000, 2000)

В программе на чистом Python в качестве матриц используйте вложенные списки. Вложенные списки можно инициализировать так:

In [ ]:
import random

a = []
for _ in range(1000):
    row = []
    for _ in range(2000):
        row.append(random.uniform())
    a.append(row)

### Мои результаты

| &nbsp;| наивный алгоритм | наивный алгоритм с транспонированием | atlas blas | numpy openblas |
| :---: | :---: | :---: | :---: | :---: |
| C | 11.6 (10) | 4.25 (10) | 0.537 (100) | 0.145 (100) |
| Python | 490 (1) | 394 (1) |  |  |
| `numpy.matmul` |  |  |  | 0.153 (100) |
| Numpy | 1854 (1) | 1951 (1) |  |  |
| Cython | 7.18 (100) | 4.96 (100) |  | 0.131 (100) |
| `ctypes` | 7.50 (10) | 4.78 (10) |  | 0.15(10) |
| JIT nopython | 611 | 536 |  |  |


## <font color=green>Библиотека Numpy</font>

Библиотека `numpy` оперирует многомерными массивами `ndarray`. Массивы Numpy не являются разновидностью списков или кортежей. В отличие от элементов списка, у элементов массива должен быть одинаковый тип данных. В сравнении со списками массивы поддерживают больше способов обращения к своим элементам.

In [ ]:
import numpy as np

z = np.zeros([3, 4])
print(z)
print(z[1, 2])
print(z[2])
print(z[:, 1])
print(z[:2, -2:])

Библиотека Numpy устанавливается с помощью `pip`. В терминале обязательно должен быть доступен интерпретатор. Интерпретатор запускается одной из команд:
```bash
py  
python
python3
```
Обязательно убедитесь, что запустился интерпретатор нужной версии (нам подходят 3.5+).
Далее для выбранного интерпретатора надо установить Numpy.
```bash
py -m pip install numpy
```